In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 請輸入資料夾之所在位置
FOLDERNAME = 'Colab\ Notebooks/SC201_L13'
assert FOLDERNAME is not None, "[!] Enter the foldername."

In [ ]:
%cd drive/MyDrive/$FOLDERNAME
BATCH_SIZE = 64

In [ ]:
# Calculate the mean/std for Training Dataset
# You will see the mean/std of your data at the end of this cell
"""
import torchvision.datasets as dset
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torch

transform = T.Compose([T.Resize((64, 64)), T.ToTensor()])
dataset = dset.ImageFolder('train/', transform=transform)
loader = DataLoader(dataset, batch_size=BATCH_SIZE)
mean = 0.0
std = 0.0
n_samples = 0
for batch, _ in loader:
  # Rearrange batch to be the shape of [B, C, W * H]
  batch = batch.view(batch.size(0), batch.size(1), -1)
  # Update total number of images
  n_samples += batch.size(0)
  # Compute mean and std here
  mean += batch.mean(2).sum(0) 
  std += batch.std(2).sum(0)
mean /= n_samples
std /= n_samples
print('mean on Training:', mean)
print('std on Training:', std)
"""

In [ ]:
import torch
mean = torch.tensor([0.4841, 0.4508, 0.4127])
std = torch.tensor([0.2155, 0.2103, 0.2101])
# Data Pre-processing by PyTorch
##################
#                #
#      TODO:     #
#                #      
##################

In [ ]:
%ls
# Load Your Data by PyTorch
##################
#                #
#      TODO:     #
#                #      
##################

In [ ]:
# Create Mini-batches by PyTorch
mini_trains = None
##################
#                #
#      TODO:     #
#                #      
##################

In [ ]:
# See if GPU is available
# To turn GPU on: Edit -> Notebook Setting -> GPU

import torch

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

In [ ]:
"""
Plot some training data! 
We need to denormalize PyTorch Tensor first, 
and then change it to images with RGB channel
"""

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
classes = ['cat','dog']

# Reverse the normalized value to 0-255
def denormalize(image):
  image = T.Normalize(-mean/std,1/std)(image) #denormalize
  image = image.permute(1,2,0) #Changing from 3x224x224 to 224x224x3
  image = torch.clamp(image,0,1)
  return image

# Display an image
def imshow(img):
    img = denormalize(img) 
    plt.imshow(img)

dataiter = iter(mini_trains)
images, labels = dataiter.next()
# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 8))
# display 20 images
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title("{} ".format( classes[labels[idx]]))

In [ ]:
import torch.nn as nn

In [ ]:
model = nn.Sequential(
    # Your Model Here
    ##################
    #                #
    #      TODO:     #
    #                #      
    ##################
    pass
)

In [ ]:
from torchsummary import summary
summary(model.cuda(), (3, 64, 64))

In [ ]:
import torch.optim as optim
learning_rate = 1e-3
# Choose your optimizer
##################
#                #
#      TODO:     #
#                #      
##################

In [ ]:
"""
Training Process begins!
We will be using your model and optimizer to 
call forward pass / backward pass.
However, which loss function should you choose?
Let's try out! You will see the difference between different loss function in decreasing training cost
"""

In [ ]:
from tqdm.notebook import tqdm

def train():
  
  # Turn on the training mode on your model
  model.train()
  
  for data, target in tqdm(mini_trains):  
    
    # move tensors to GPU if CUDA is available
    data = data.to(device)
    target = target.to(device)

    # clear the gradients of all optimized variables
    optimizer.zero_grad()
    
    # forward pass: compute predicted outputs by passing inputs to the model
    scores = model(data)
    
    # calculate the batch loss
    # Choose your loss function!
    ##################
    #                #
    #      TODO:     #
    #                #      
    ##################

    loss.backward()

    # perform a single optimization step (parameter update)
    optimizer.step()

In [ ]:
def evaluate_predictor():

  # Turn on evaluation mode on your model
  model.eval()

  train_loss = 0
  train_acc = 0
  for data, target in tqdm(mini_trains):
    # move tensors to GPU if CUDA is available
    data = data.to(device)
    target = target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    scores = model(data)
    
    # Choose your loss function!
    ##################
    #                #
    #      TODO:     #
    #                #      
    ##################

    train_loss += loss.item()*data.size(0)
    prediction = scores.data.max(1)[1]
    acc = prediction.eq(target).sum().item()
    train_acc += acc

	# calculate average losses
  train_loss = train_loss / len(mini_trains.dataset)
  train_acc = train_acc / len(mini_trains.dataset)
  return train_loss, train_acc

In [ ]:
for epoch in range(7):
	print('EPOCH: {}'.format(epoch+1))
	train()
	train_loss, train_acc = evaluate_predictor()

	print('\tTraining Loss: {:.6f}\tTraining Acc: {:.6f}'.format(train_loss, train_acc))

In [ ]:
"""
Loading Test Data
-------------------------
"""
test_data = dset.ImageFolder('test/', transform=transform)
print(test_data)
mini_tests = DataLoader(test_data, batch_size = 64, shuffle = True)

In [ ]:
model.eval()
test_acc = 0
for data, target in tqdm(mini_tests):
  data = data.to(device)
  target = target.to(device)
  scores = model(data)
  prediction = scores.data.max(1)[1]
  acc = prediction.eq(target).sum().item()
  test_acc += acc

# calculate average losses
test_acc = test_acc / len(mini_tests.dataset)
print('Test Acc:', test_acc)